In [ ]:
import numpy as np
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import collections
from tqdm import tqdm_notebook as tqdm
from datetime import datetime

from math import ceil, floor, log
import cv2

import tensorflow as tf
import keras

import sys

# from keras_applications.resnet import ResNet50
from keras.applications import *
from keras.layers import *
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib
from tensorflow import keras
from PIL import Image
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
from keras.callbacks import TensorBoard
#from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
#from sklearn.model_selection import train_test_split
#from tqdm import tqdm
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

input_path = "../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/"
test_images_dir = input_path + 'stage_2_test/'
train_images_dir = input_path + 'stage_2_train/'

In [ ]:
path = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/ID_000012eaf.dcm'
img = pydicom.dcmread(path)
img

In [ ]:
def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(dcm, window_center, window_width):
    
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)
    
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)

    return img

def bsb_window(dcm):
    brain_img = window_image(dcm, 40, 80)
    subdural_img = window_image(dcm, 80, 200)
    soft_img = window_image(dcm, 40, 380)
    
    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380
    bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)

    return bsb_img

# Sanity Check
# Example dicoms: ID_2669954a7, ID_5c8b5d701, ID_52c9913b1

dicom = pydicom.dcmread(train_images_dir + 'ID_5c8b5d701' + '.dcm')
#                                     ID  Label
# 4045566          ID_5c8b5d701_epidural      0
# 4045567  ID_5c8b5d701_intraparenchymal      1
# 4045568  ID_5c8b5d701_intraventricular      0
# 4045569      ID_5c8b5d701_subarachnoid      1
# 4045570          ID_5c8b5d701_subdural      1
# 4045571               ID_5c8b5d701_any      1
plt.imshow(bsb_window(dicom), cmap=plt.cm.bone);


In [ ]:
def window_with_correction(dcm, window_center, window_width):
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img

def window_without_correction(dcm, window_center, window_width):
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img

def window_testing(img, window):
    brain_img = window(img, 40, 80)
    subdural_img = window(img, 80, 200)
    soft_img = window(img, 40, 380)
    
    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380
    bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)

    return bsb_img

# example of a "bad data point" (i.e. (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100) == True)
dicom = pydicom.dcmread(train_images_dir + "ID_036db39b7" + ".dcm")

fig, ax = plt.subplots(1, 2)

ax[0].imshow(window_testing(dicom, window_without_correction), cmap=plt.cm.bone);
ax[0].set_title("original")
ax[1].imshow(window_testing(dicom, window_with_correction), cmap=plt.cm.bone);
ax[1].set_title("corrected");

In [ ]:
df = pd.read_csv('../input/rsna-csv-files/RSNA_DATA/good_slices.csv',index_col = 'Unnamed: 0')


In [ ]:
df.head()

# Data Preparation

In [ ]:
from tqdm import trange
def get_partition_labels(df,frac = 0.25):
    partition = dict()
    labels = dict()
    df = df.sample(frac = frac, random_state= 1)
    for i in trange(len(df)):
        id_ = df.Image.iloc[i]
        #print('id = ',id_,type(id_))
        label = df.iloc[i,1:7].to_numpy(dtype = 'int32')
        #print('label =',label)
        labels[id_] = label
    
    training = df.sample(frac = 0.6,random_state= 1)
    validation = df.drop(training.index, axis = 0)
    test = validation.sample(frac = 0.5,random_state= 1)
    validation = validation.drop(test.index, axis = 0) 
    partition['train'] = list(training.Image)
    partition['validation'] = list(validation.Image)
    partition['test'] = list(test.Image)
    return partition,labels

In [ ]:
partition,labels = get_partition_labels(df,frac = 1)

In [ ]:
def _read(path, desired_size):
    """Will be used in DataGenerator"""
    
    dcm = pydicom.dcmread(path)
    
    try:
        img = bsb_window(dcm)
    except:
        img = np.zeros(desired_size)
    
    
    img = cv2.resize(img, desired_size[:2], interpolation=cv2.INTER_LINEAR)
    return img

# Another sanity check 
plt.imshow(
    _read(train_images_dir+'ID_5c8b5d701'+'.dcm', (256, 256,3)), cmap=plt.cm.bone
);

# Loader

In [ ]:
import numpy as np
import keras
#Data loader specialized 
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        if self.labels is not None:
            X, y = self.__data_generation(list_IDs_temp)
            return X, y
        else:
            X = self.__data_generation(list_IDs_temp)
            return X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim),dtype = 'float32') #,self.n_channels))
        if self.labels is not None: # training phase
            #Use self.n_classes instead of 6 
            y = np.empty((self.batch_size,6), dtype='float32')
            #print(y.shape)
            #print(X.shape)
            # Generate data
            for i, ID in enumerate(list_IDs_temp):
                # Store sample
                image_dir = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/'
                X[i,] = _read(image_dir+ID+'.dcm',self.dim)
                #print(X)
                # Store class
                y[i] = self.labels[ID]
                #print(y[i])
            return X,y
        
        else: # test phase
            for i, ID in enumerate(list_IDs_temp):
                path = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_test/'
                X[i,] = _read(path+ID+'.dcm',self.dim)
            
            return X

# Loss functions

In [ ]:
from tensorflow.keras import backend as K

def weighted_log_loss(y_true, y_pred):
    """
    Can be used as the loss function in model.compile()
    ---------------------------------------------------
    """
    
    class_weights =  tf.constant([2., 1., 1., 1., 1., 1.])
    
    eps = tf.keras.backend.epsilon()
    
    
    y_pred = tf.clip_by_value(y_pred, eps, 1.0-eps)

    out = -(         y_true  * tf.math.log(      y_pred) * class_weights
            + (1.0 - y_true) * tf.math.log(1.0 - y_pred) * class_weights)
    
    return tf.reduce_mean(out, axis=-1)


####
def weighted_log_loss_V2(y_true, y_pred):
    """
    Can be used as the loss function in model.compile()
    ---------------------------------------------------
    """
    
    class_weights =  tf.constant([2., 1., 1., 1., 1., 1.])
    
    eps = tf.keras.backend.epsilon()
    
    
    y_pred = tf.clip_by_value(y_pred, eps, 1.0-eps)

    out = -(         y_true  * tf.math.log(      y_pred) * class_weights
            + (1.0 - y_true) * tf.math.log(1.0 - y_pred) * class_weights)
    
    return tf.reduce_mean(out, axis=-1)
####

def _normalized_weighted_average(arr, weights=None):
    """
    A simple Keras implementation that mimics that of 
    numpy.average(), specifically for this competition
    """
    
    if weights is not None:
        scl = K.sum(weights)
        weights = K.expand_dims(weights, axis=1)
        return K.sum(K.dot(arr, weights), axis=1) / scl
    return K.mean(arr, axis=1)


def weighted_loss(y_true, y_pred):
    """
    Will be used as the metric in model.compile()
    ---------------------------------------------
    
    Similar to the custom loss function 'weighted_log_loss()' above
    but with normalized weights, which should be very similar 
    to the official competition metric:
        https://www.kaggle.com/kambarakun/lb-probe-weights-n-of-positives-scoring
    and hence:
        sklearn.metrics.log_loss with sample weights
    """
    
    class_weights = tf.constant([2., 1., 1., 1., 1., 1.])
    
    eps = tf.keras.backend.epsilon()
    
    y_pred = tf.clip_by_value(y_pred, eps, 1.0-eps)

    loss = -(        y_true  * tf.math.log(      y_pred)
            + (1.0 - y_true) * tf.math.log(1.0 - y_pred))
    
    loss_samples = _normalized_weighted_average(loss, class_weights)
    
    return tf.reduce_mean(loss_samples)


def weighted_log_loss_metric(trues, preds):
    """
    Will be used to calculate the log loss 
    of the validation set in PredictionCheckpoint()
    ------------------------------------------
    """
    class_weights = [2., 1., 1., 1., 1., 1.]
    
    epsilon = 1e-7
    
    preds = np.clip(preds, epsilon, 1-epsilon)
    loss = trues * np.log(preds) + (1 - trues) * np.log(1 - preds)
    loss_samples = np.average(loss, axis=1, weights=class_weights)

    return - loss_samples.mean()
'''
def log_loss(y_pred,y_true):
    #y_pred = np.clip(y_pred, 1e-7, 1.0-1e-7)
    ones = np.where(y_true)
    zeros = np.where(y_true == 0)
    likelihood = np.prod(y_pred[ones]) *  np.prod(y_pred[zeros])
    logloss = -1 * np.log(likelihood)
    return logloss
'''

def log_loss(y_pred,y_true):
    """
    Will be used to calculate the log loss 
    of the validation set in metrics
    ------------------------------------------
    """
    eps = 1e-15
    
    y_pred = tf.clip_by_value(y_pred, eps, 1.0 - eps)
    
    ones = tf.where(y_true)
    zeros = tf.where(y_true == 0)
    
    likelihood = tf.math.reduce_prod(tf.gather(y_pred,ones)) *  tf.math.reduce_prod(1 - tf.gather(y_pred,zeros))
    logloss = -1 * tf.math.log(likelihood)
    return logloss




# Training

In [ ]:
params = {'dim':(299,299,3),
         'batch_size':32,
         'n_classes':6,
         'n_channels':0,
         'shuffle':True}
# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)


# Design model
model = Sequential([
    tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=params['dim']),
    Flatten(),
    Dense(30,activation = 'relu'),
    Dense(6,activation = 'sigmoid')
])
model = Sequential([
    tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=params['dim'],
    pooling='max'),
    Flatten(),
    Dense(30,activation = 'relu'),
    Dense(6,activation = 'sigmoid')
])

fit(
    x=None, y=None, batch_size=None, epochs=1, verbose='auto',
    callbacks=None, validation_split=0.0, validation_data=None, shuffle=True,
    class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None,
    validation_steps=None, validation_batch_size=None, validation_freq=1,
    max_queue_size=10, workers=1, use_multiprocessing=False
)

# Callbacks

In [ ]:
#Callbacks
#%load_ext tensorboard
#%tensorboard --logdir logs

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
if 'checkpoint' not in os.listdir('./'):
    os.mkdir('./checkpoint')
ckpt = './checkpoint/cp.ckpt'
my_callbacks = [
    ModelCheckpoint(filepath = ckpt, monitor = 'val_loss' ,save_best_only=True,verbose = 3),
    ReduceLROnPlateau(monitor= 'val_loss', factor=0.1, patience= 2, verbose=1,mode='auto', min_delta=0.0001)]


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[weighted_loss,log_loss,keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC(num_thresholds=200, curve='ROC',summation_method='interpolation', multi_label=True, label_weights=[2, 1, 1, 1, 1, 1])])

In [ ]:
epochs = 4
# Train model on dataset
model.fit(training_generator,
         validation_data=validation_generator,
         epochs=epochs,
         workers = 4, 
         use_multiprocessing= True,
         callbacks = my_callbacks)

fit(
    x=None, y=None, batch_size=None, epochs=1, verbose='auto',
    callbacks=None, validation_split=0.0, validation_data=None, shuffle=True,
    class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None,
    validation_steps=None, validation_batch_size=None, validation_freq=1,
    max_queue_size=10, workers=1, use_multiprocessing=False
)

In [ ]:
model.save('./model.h5') 

In [ ]:
#Need if you use a pretrained model
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[weighted_loss,log_loss,keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC(num_thresholds=200, curve='ROC',summation_method='interpolation', multi_label=True, label_weights=[2, 1, 1, 1, 1, 1])])


params = {'dim':(299,299,3),
         'batch_size':32,
         'n_classes':6,
         'n_channels':0,
         'shuffle':False}



test_generator = DataGenerator(partition['test'], labels, **params)
model.evaluate(test_generator,verbose = 1, use_multiprocessing = True, workers = 4)

from tqdm import tqdm
y_true = []
for labels in tqdm(test_generator):
    batch = labels[1]
    for label in batch:
        y_true.append(label)

y_true = np.array(y_true)

test_preds = model.predict(test_generator,workers=4, use_multiprocessing=True, verbose = 1)

import copy
y_pred_copy = copy.deepcopy(test_preds)
y_pred=y_pred_copy
y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = 0

from sklearn.metrics import multilabel_confusion_matrix
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
cm_multi = multilabel_confusion_matrix(y_true, y_pred, sample_weight=None, labels=None)

for i,cm in enumerate(cm_multi):
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=None)
    disp.plot() 
  

evaluate(
    x=None, y=None, batch_size=None, verbose=1, sample_weight=None, steps=None,
    callbacks=None, max_queue_size=10, workers=4, use_multiprocessing=False,
    return_dict=False, **kwargs
)

# Submission Part

In [ ]:
def get_test_dcms():
    path = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_sample_submission.csv'
    df1 = pd.read_csv(path)
    df1["Image"] = df1["ID"].str.slice(stop=12)
    df1["Diagnosis"] = df1["ID"].str.slice(start=13)
    df1 = df1.loc[:, [ "Diagnosis", "Image","Label"]]
    df1 = df1.pivot(index = 'Image', columns = 'Diagnosis', values = 'Label')
    return list(df1.index)

In [ ]:
def get_pred(path = '../input/checkpoint'):
    params = {'dim':(299,299,3),
         'batch_size':16,
         'n_classes':6,
         'n_channels':0,
         'shuffle':True}
    dcms = get_test_dcms() #121232 element
    test_generator = DataGenerator(dcms,labels = None,**params)
    #model = tf.keras.models.load_model( path, compile = False)
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[weighted_loss,log_loss,keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC(num_thresholds=200, curve='ROC',summation_method='interpolation', multi_label=True, label_weights=[2, 1, 1, 1, 1, 1])])
    #model = model.load_weights(path)
    preds = model.predict(test_generator, workers=4, use_multiprocessing=True,verbose = 1)
    return preds

predict(
    x, batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10,
    workers=1, use_multiprocessing=False
)

In [ ]:
preds = get_pred(path = './checkpoint')

In [ ]:
def get_submission(preds):
    dcms = get_test_dcms()
    deneme = pd.DataFrame(preds, columns=[ 'epidural', 'intraparenchymal', 'intraventricular','subarachnoid', 'subdural','any'], index=dcms)
    values = deneme.stack().values
    path = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_sample_submission.csv'
    submission = pd.read_csv(path)
    submission.drop_duplicates()
    submission.Label = values
    #print(submission.head())
    submission.to_csv('./submission.csv',index = None)
    return submission
    
    

In [ ]:
get_submission(preds)